In [1]:
import torch
import os
import cv2
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
from torchvision import transforms
from tqdm import tqdm
PROJECT_NAME = "Intel-Classification-V3"
device = 'cuda'

In [2]:
transformation_data = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

In [3]:
img = cv2.imread('./data/buildings/0.jpg')

In [4]:
img_pil_image = Image.fromarray(img)
# transformation_data(img_pil_image)

In [5]:
np.eye(5,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [6]:
def load_image(data_dir="./data/"):
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for label in os.listdir(data_dir):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label
    for directory in tqdm(os.listdir(data_dir)):
        for file_dir in os.listdir(data_dir + directory):
            img = cv2.imread(data_dir + directory + "/" + file_dir)
            img = cv2.resize(img,(56,56))
            data.append([np.array(transformation_data(Image.fromarray(img))),np.eye(labels[directory]+1,idx+1)[-1]])
    np.random.shuffle(data)
    data = data[:5000]
    X = []
    y = []
    for d_iter in data:
        X.append(d_iter[0])
        y.append(d_iter[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=True)
    X_train = torch.from_numpy(np.array(X_train)).to(device)
    y_train = torch.from_numpy(np.array(y_train)).to(device)
    X_test = torch.from_numpy(np.array(X_test)).to(device)
    y_test = torch.from_numpy(np.array(y_test)).to(device)
    return labels,labels_r,data,X,y,X_train,X_test,y_train,y_test,idx

In [7]:
labels,labels_r,data,X,y,X_train,X_test,y_train,y_test,idx = load_image()

In [8]:
torch.save(labels,'./save/labels.pt')
torch.save(labels,'./save/labels.pth')
torch.save(labels_r,'./save/labels_r.pt')
torch.save(labels_r,'./save/labels_r.pth')
torch.save(data,'./save/data.pt')
torch.save(data,'./save/data.pth')
torch.save(X,'./save/X.pt')
torch.save(y,'./save/y.pth')
torch.save(X_train,'./save/X_train.pt')
torch.save(X_test,'./save/X_test.pth')
torch.save(y_train,'./save/y_train.pt')
torch.save(y_test,'./save/y_test.pth')

In [9]:
len(X_train),len(X_test),len(y_train),len(y_test)

(3750, 1250, 3750, 1250)

In [10]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [11]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for y_iter,pred in zip(y,preds):
        y_iter = torch.argmax(y_iter)
        pred = torch.argmax(pred)
        if pred == y_iter:
            correct += 1
        total += 1
    return round(correct/total,3)

In [12]:
from torch.nn import *

In [13]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,6)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        preds = preds.view(-1,128*3*3)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [14]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [15]:
import wandb

In [16]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        try:
            X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
            y_batch = y_train[idx:idx+batch_size].to(device)
            preds = model(X_batch)
            loss = criterion(preds.double(),y_batch.double())
            optimizer.zero_grad()
            optimizer.step()
            loss.backward()
            wandb.log({
                "Test Loss":get_loss(model,X_test,y_test,criterion),
                "Test Accuracy":get_accuracy(model,X_test,y_test),
                "Accuracy":get_accuracy(model,X_batch,y_batch),
                "Loss":get_loss(model,X_batch,y_batch,criterion)
            })
        except:
            pass

In [17]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,6)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        preds = preds.view(-1,128*3*3)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [18]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [19]:
import wandb

In [20]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
            X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
            y_batch = y_train[idx:idx+batch_size].to(device)
            preds = model(X_batch)
            loss = criterion(preds.double(),y_batch.double())
            optimizer.zero_grad()
            optimizer.step()
            loss.backward()
            wandb.log({
                "Test Loss":get_loss(model,X_test,y_test,criterion),
                "Test Accuracy":get_accuracy(model,X_test,y_test),
                "Accuracy":get_accuracy(model,X_batch,y_batch),
                "Loss":get_loss(model,X_batch,y_batch,criterion)
            })

In [21]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,6)
        self.relu = nn.ReLU(inplace=False)
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        preds = preds.view(-1,128*3*3)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [22]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [23]:
import wandb

In [24]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
            X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
            y_batch = y_train[idx:idx+batch_size].to(device)
            preds = model(X_batch)
            loss = criterion(preds.double(),y_batch.double())
            optimizer.zero_grad()
            optimizer.step()
            loss.backward()
            wandb.log({
                "Test Loss":get_loss(model,X_test,y_test,criterion),
                "Test Accuracy":get_accuracy(model,X_test,y_test),
                "Accuracy":get_accuracy(model,X_batch,y_batch),
                "Loss":get_loss(model,X_batch,y_batch,criterion)
            })

Accuracy,▁
Loss,▁
Test Accuracy,▁
Test Loss,▁
Accuracy,0.281
Loss,1.78339
Test Accuracy,0.166
Test Loss,1.79101


In [25]:
preds.shape

torch.Size([32, 6])

In [26]:
y_batch.shape

torch.Size([32, 6])

In [27]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
            X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
            y_batch = y_train[idx:idx+batch_size].to(device)
            preds = model(X_batch)
            loss = criterion(preds,y_batch)
            optimizer.zero_grad()
            optimizer.step()
            loss.backward()
            wandb.log({
                "Test Loss":get_loss(model,X_test,y_test,criterion),
                "Test Accuracy":get_accuracy(model,X_test,y_test),
                "Accuracy":get_accuracy(model,X_batch,y_batch),
                "Loss":get_loss(model,X_batch,y_batch,criterion)
            })